In [ ]:
import plotly.graph_objects as go
import pandas as pd
import yfinance as yf
import time
import numpy as np
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
from IPython.display import clear_output, display

# Optimized Indicator Settings
SMA_SHORT = 14  
SMA_LONG = 28   
MACD_SHORT_WINDOW = 15  
MACD_LONG_WINDOW = 30    
MACD_SIGNAL_WINDOW = 10  
RSI_OVERBOUGHT = 75  
RSI_OVERSOLD = 25    
ATR_MULTIPLIER = 3  
RF_N_ESTIMATORS = 300  
RF_MAX_DEPTH = 15  
TRAIN_TEST_SPLIT = 0.25  

# Fetch historical forex data
def get_historical_data(interval="5m", period="60d"):
    gbpusd = yf.Ticker("GBPUSD=X")
    data = gbpusd.history(period=period, interval=interval)
    return data

# Fetch data for training (1-year, 1-day interval)
def get_model_training_data():
    return get_historical_data(interval="1d", period="1y")

# Fetch data for chart display (60 days, 5-minute interval)
def get_chart_data():
    return get_historical_data(interval="5m", period="60d")

# Compute RSI
def compute_rsi(series, period=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

# Compute MACD and Signal Line
def compute_macd(series):
    short_ema = series.ewm(span=MACD_SHORT_WINDOW, adjust=False).mean()
    long_ema = series.ewm(span=MACD_LONG_WINDOW, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=MACD_SIGNAL_WINDOW, adjust=False).mean()
    return macd, signal

# Compute ATR (Average True Range) for volatility filtering
def compute_atr(data, period=14):
    high_low = data['High'] - data['Low']
    high_close = abs(data['High'] - data['Close'].shift())
    low_close = abs(data['Low'] - data['Close'].shift())
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    return true_range.rolling(period).mean()

# Feature Engineering
def feature_engineering(data):
    data['Return'] = data['Close'].pct_change()
    data['SMA_Short'] = data['Close'].rolling(window=SMA_SHORT).mean()
    data['SMA_Long'] = data['Close'].rolling(window=SMA_LONG).mean()
    data['Volatility'] = data['Close'].rolling(window=20).std()
    data['High_Low_Range'] = data['High'] - data['Low']
    data['Momentum'] = data['Close'] - data['Close'].shift(4)
    data['Breakout'] = (data['Close'] > data['Close'].rolling(window=20).max()).astype(int)
    data['RSI'] = compute_rsi(data['Close'])
    data['MACD'], data['Signal_Line'] = compute_macd(data['Close'])
    data['ATR'] = compute_atr(data)
    data.dropna(inplace=True)
    return data

# Prepare Data for Training
def prepare_ml_data(data):
    feature_columns = ['Return', 'SMA_Short', 'SMA_Long', 'Volatility', 'High_Low_Range', 'Momentum', 'Breakout', 'RSI', 'MACD', 'Signal_Line', 'ATR']
    X = data[feature_columns]
    y = np.where(data['Close'].shift(-1) > data['Close'], 1, 0)  
    return X, y

# Train the ML Model
def train_ml_model():
    print("Fetching historical data for training...")
    data = get_model_training_data()
    
    if data is None or data.empty:
        print("Error: No data fetched.")
        return
    
    print("Performing feature engineering...")
    data = feature_engineering(data)
    
    print("Preparing training data...")
    X, y = prepare_ml_data(data)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TRAIN_TEST_SPLIT, random_state=42)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    print("Training model...")
    model = RandomForestClassifier(n_estimators=RF_N_ESTIMATORS, max_depth=RF_MAX_DEPTH, random_state=42)
    model.fit(X_train, y_train)
    
    accuracy = model.score(X_test, y_test)
    print(f"Model trained with accuracy: {accuracy:.2f}")
    
    joblib.dump(model, "forex_model.pkl")
    joblib.dump(scaler, "forex_scaler.pkl")
    print("Model and scaler saved successfully.")
    return model, scaler

# Analyze & Generate Trading Signals
def analyze_and_generate_signals(data, model, scaler):
    if data is None or data.empty or len(data) < 20:
        return None, None
    
    data = feature_engineering(data)
    latest_data = data.iloc[-1:][['Return', 'SMA_Short', 'SMA_Long', 'Volatility', 'High_Low_Range', 'Momentum', 'Breakout', 'RSI', 'MACD', 'Signal_Line', 'ATR']]
    latest_data = scaler.transform(latest_data)

    prediction = model.predict(latest_data)[0]
    if data['ATR'].iloc[-1] > ATR_MULTIPLIER * data['ATR'].mean():
        return "🚫 Market too volatile, no trade", None  

    signal_type = "CALL 🟩" if prediction == 1 else "PUT 🟥"
    current_time = datetime.utcnow() - timedelta(hours=3)
    trade_time = (current_time + timedelta(minutes=5)).strftime("%H:%M")
    signal_text = f"💰 5-minute expiry\nGBP/USD;{trade_time};{signal_type}\n🕐 TIME TO {trade_time}"
    return signal_text, trade_time

# Create Candlestick Chart
def create_candlestick_chart(data, title):
    if data is None or data.empty:
        print(f"No data to display for {title}.")
        return None
    
    fig = go.Figure(data=[go.Candlestick(x=data.index,
                                         open=data['Open'],
                                         high=data['High'],
                                         low=data['Low'],
                                         close=data['Close'])])
    fig.update_layout(title=title, xaxis_title='Time', yaxis_title='Price', xaxis_rangeslider_visible=False)
    return fig

# Update Charts & Generate Signals
def update_charts_and_signals():
    model, scaler = train_ml_model()
    if model is None:
        print("Model training failed.")
        return
    
    signals_history = []
    try:
        while True:
            data_5min = get_chart_data()
            fig_5min = create_candlestick_chart(data_5min, "GBP/USD 5-Minute Chart")
            
            if fig_5min is not None:
                signal_text, trade_time = analyze_and_generate_signals(data_5min, model, scaler)
                if signal_text:
                    signals_history.append(signal_text)
                    signals_history = signals_history[-5:]
                    clear_output(wait=True)
                    display(fig_5min)
                    print("\n".join(signals_history))
            
            time.sleep(300)  # Update every 5 minutes

    except KeyboardInterrupt:
        print("Chart update stopped.")

if __name__ == "__main__":
    update_charts_and_signals()


💰 5-minute expiry
GBP/USD;08:31;CALL 🟩
🕐 TIME TO 08:31
